# **Data Collection**

## Objectives

* Fetch and prepare the data from Kaggle and prepare for processing


## Inputs

* Json Kaggle file - authentication

## Outputs

* Generate input and output for cherry images

In [19]:
%pip install -r /workspace/cherry/requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [20]:
import numpy
import os

## Change working directory

In [21]:
current_dir = os.getcwd()
current_dir

'/workspace/cherry'

In [22]:
os.chdir('/workspace/cherry')

In [23]:
current_dir = os.getcwd()
current_dir

'/workspace/cherry'

install Kaggle

In [24]:
#install kaggle
%pip install kaggle==1.5.12

Note: you may need to restart the kernel to use updated packages.


*Run cell ** To change directory and set permissions for the Kaggle authentication key **

In [26]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

In [28]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherry_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

 96%|████████████████████████████████████▌ | 53.0M/55.0M [00:01<00:00, 42.9MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:01<00:00, 30.6MB/s]


In [29]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

# Data preperation

## Data cleaning

** remove non image files

In [30]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir) 
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        
            #print(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location) # remove non image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file",len(j))
        print(f"Folder: {folder} - has non-image file",len(i))

In [32]:
remove_non_image_file(my_data_dir='inputs/cherry_dataset/cherry-leaves')

IsADirectoryError: [Errno 21] Is a directory: 'inputs/cherry_dataset/cherry-leaves/train/healthy'

## Split train validation test set

In [33]:
import os
import random
import shutil
import joblib

def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):
  
  if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
    print("train_set_ratio + validation_set_ratio + test_set_ratio should sum 1.0")
    return

  # gets classes labels
  labels = os.listdir(my_data_dir) # it should get only the folder name
  if 'test' in labels:
    pass
  else: 
    # create train, test folders with classess labels sub-folder
    for folder in ['train','validation','test']:
      for label in labels:
        os.makedirs(name=my_data_dir+ '/' + folder + '/' + label)

    for label in labels:

      files = os.listdir(my_data_dir + '/' + label)
      random.shuffle(files)

      train_set_files_qty = int(len(files) * train_set_ratio)
      validation_set_files_qty = int(len(files) * validation_set_ratio)

      count = 1
      for file_name in files:
        if count <= train_set_files_qty:
          # move given file to train set
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                      my_data_dir + '/train/' + label + '/' + file_name)
          

        elif count <= (train_set_files_qty + validation_set_files_qty ):
          # move given file to validation set
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                      my_data_dir + '/validation/' + label + '/' + file_name)

        else:
          # move given file to test set
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                  my_data_dir + '/test/' +label + '/'+ file_name)
          
        count += 1

      os.rmdir(my_data_dir + '/' + label)

"* The training set is divided into 0.70 ratio of data.\n",
        "* The validation set is divided into 0.10 ratio of data.\n",
        "* The test set is divided into 0.20 ratio of data."

In [34]:
split_train_validation_test_images(my_data_dir = f"inputs/cherry_dataset/cherry-leaves",
                        train_set_ratio = 0.7,
                        validation_set_ratio=0.1,
                        test_set_ratio=0.2
                        )